# Neural Network tester
- Load a pre-trained NN and test it against 1296 passwords
- Save a CSV with the hystory and cuts (it could be used by *matches_analyzer.ipynb*)

In [1]:
import tensorflow as tf
import pandas as pd
import json
import os

-----------------------

In [3]:
PRINT_INFO = False

NETWORKS_FAMILY_PATH = "../4_training/trained/hopeful/model_20191026_1947"

In [7]:
networks_name = os.path.basename(NETWORKS_FAMILY_PATH)
networks_paths = os.listdir(NETWORKS_FAMILY_PATH)
assert len(networks_paths) == 4
networks_paths.sort(key= lambda name: int(name[-1]))
print("Networks to test:{}".format(networks_paths))

MODELS_PATH = []
INFOS_PATH = []
for network_path in networks_paths:
    model_path = NETWORKS_FAMILY_PATH+"/"+network_path+"/export/predict/"
    info_path = NETWORKS_FAMILY_PATH+"/"+network_path+"/model.info"
    export_id = os.listdir(model_path)
    model_path += export_id[0]
    MODELS_PATH.append(model_path)
    INFOS_PATH.append(info_path)
print('Network(s) name: '+networks_name)

Networks to test:['model_20191026_1947_0', 'model_20191026_1947_1', 'model_20191026_1947_2', 'model_20191026_1947_3']
Network(s) name: model_20191026_1947


---------------------------------------------
## Load the networks

In [8]:
with open(INFOS_PATH[0]) as f:
    nn_info = json.load(f)
    print("Info network 0:"+json.dumps(nn_info, indent=4, sort_keys=True))

Info network 0:{
    "ALPHA": 0.001,
    "ATTENTION_SIZE": 256,
    "BATCH_EVAL": 32,
    "BATCH_TRAIN": 32,
    "BW_FORGET_BIAS": 1,
    "CSV_PATH_EVAL": "../2_database/dbML/hopeful/hopeful_2019-10-05_0.8_206957_eval.csv",
    "CSV_PATH_TRAIN": "../2_database/dbML/hopeful/hopeful_2019-10-05_0.8_829843_train.csv",
    "DROP_REPRESENTATION": 0.3,
    "EPOCHS": 10,
    "EVAL_STEPS": null,
    "FW_FORGET_BIAS": 1,
    "GOAL": "goalReduced",
    "GOAL_POS": 0,
    "INPUT_ENCODING": "Peg: Hot encoding, Tips: One Hot Encoding",
    "LSTM_SIZE": 256,
    "MAXOUT_SIZE": 256,
    "MODEL_DIR_PATH": "model_20191026_1947_0",
    "MULTI_THREADING": true,
    "NUM_EVALS": 10,
    "SHUFFLE_EVAL": false,
    "_NETWORK_TYPE": "BiLSTM_AttentionSystem",
    "loss_fn": "mean_pairwise_squared_error",
    "train_duration": "3:08:47"
}


In [9]:
def load_model(model_path):
    predictor_fn = tf.contrib.predictor.from_saved_model(
                            export_dir = model_path,
                            signature_def_key="prediction")
    return predictor_fn

models = []
for path in MODELS_PATH:
    model = load_model(path)
    models.append(model)


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.loader.load or tf.compat.v1.saved_model.load. There will be a new function for importing SavedModels in Tensorflow 2.0.
Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from ../4_training/trained/hopeful/model_20191026_1947/model_20191026_1947_0/export/predict/1572123393/variables/variables
INFO:tensorflow:Restoring parameters from ../4_training/trained/hopeful/model_20191026_1947/model_20191026_1947_1/export/predict/1572134988/variables/variables
INFO:tensorflow:Restoring parameters from ../4_training/trained/hopeful/model_20191026_1947/model_20191026_1

------------------
## Test the network

In [10]:
from itertools import product
from string import ascii_uppercase
FEATURES_NAME = ['Guess 1', 'Guess 2', 'Guess 3', 'Guess 4', 'Guess 5',
          'Guess 6', 'Guess 7', 'Guess 8', 'Guess 9', 'Guess 10']

TARGET_NAME = 'PASSWORD'

HEADER = FEATURES_NAME + [TARGET_NAME]

HOT_ENCODING = {
    "100000" : 'A',
    "010000" : 'B',
    "001000" : 'C',
    "000100" : 'D',
    "000010" : 'E',
    "000001" : 'F',
}

MATCH_LOST_PAD = 'XXXX'

# Classic Mastermind rules
PSW_CARDINALITY = 6
PSW_LEN = 4
PSW_POOL_COMPLETE = [''.join(p) for p in product(ascii_uppercase[0:PSW_CARDINALITY], repeat=PSW_LEN)]

# Analysis persistance
def save_to_csv(matches, tail_name):
    df = pd.DataFrame(data=matches, columns=HEADER)
    with open('./matchesPlayed/{}_{}.csv'.format(networks_name, tail_name),'w') as f:
        df.to_csv(f, sep=',', encoding='utf-8',index=False, header=True, float_format="%.0f")

In [11]:
def change_char(s, p, r):
    return s[:p]+r+s[p+1:]

def hints_calculator(psw_goal, psw_guess):
    black_pegs = len([goal_element for goal_element, guess_element in zip(psw_goal, psw_guess) if goal_element == guess_element])
    white_pegs = sum([min(psw_goal.count(element), psw_guess.count(element)) for element in set('ABCDEF')]) - black_pegs
    return black_pegs, white_pegs

def get_clean_match():
    match = {}
    cuts = {}
    for guess in FEATURES_NAME:
        match[guess] = ["<pad>"]
        cuts[guess] = 0
    match[TARGET_NAME] = ["----"]
    cuts[TARGET_NAME] = "----"
    return match, cuts

def make_a_guess(match, psw):
    for idx, model in enumerate(models):
        prediction = model(match)['Prediction'][0]
        prediction = "".join(str(int(digit)) for digit in prediction)
        prediction = HOT_ENCODING[prediction]
        match[TARGET_NAME][0] = change_char(match[TARGET_NAME][0], idx, prediction)
    blk_peg, wht_peg = hints_calculator(psw, match[TARGET_NAME][0])
    guess = "{}{}{}".format(match[TARGET_NAME][0], blk_peg, wht_peg)
    match[TARGET_NAME][0] = '----'
    won = blk_peg == 4
    return guess, won, (blk_peg, wht_peg)

def guess_pool_pruning(psw_pool, next_guess, hints):
    to_remove = []
    for psw in psw_pool:
        if hints != hints_calculator(psw, next_guess):
            to_remove.append(psw)
    len_psw_pool = len(psw_pool)
    psw_pool.difference_update(to_remove)
    return int(len_psw_pool - len(psw_pool))

In [12]:
from pprint import pprint

victories = 0
matches = []
cuts_hist = []

for PSW in PSW_POOL_COMPLETE:
    won = False
    psw_pool = set(PSW_POOL_COMPLETE.copy())
    match, cuts = get_clean_match()
    for guess_pos in FEATURES_NAME:
        guess, is_psw, pegs = make_a_guess(match, PSW)
        match[guess_pos] = [guess]
        cuts[guess_pos] = guess_pool_pruning(psw_pool, guess[:-2], pegs)
        if is_psw:
            won = True
            break
            
    match = {guess_pos: guess[0] for guess_pos, guess in match.items()}
    match[TARGET_NAME] = PSW
    cuts[TARGET_NAME] = PSW
    if won:
        victories = victories + 1
    else:
        match['Guess 10'] = MATCH_LOST_PAD
        if PRINT_INFO:
            print("Psw {} not guessed, match played:\n".format(PSW))
            pprint(match)
            print('\n\n')
    matches.append(match)
    cuts_hist.append(cuts)

In [13]:
save_to_csv(matches, 'play')
save_to_csv(cuts_hist, 'play_cuts')

In [14]:
print("Victories: {} / {}".format(victories, len(PSW_POOL_COMPLETE)))
print("Ratio: {}".format(victories/len(PSW_POOL_COMPLETE)))

Victories: 1180 / 1296
Ratio: 0.9104938271604939
